In [2]:
import torch
import torch.nn as nn

import numpy as np

import random



random_seed = 42



torch.manual_seed(random_seed)

torch.backends.cudnn.deterministic = True

torch.backends.cudnn.benchmark = False



np.random.seed(random_seed)

random.seed(random_seed)



torch.cuda.manual_seed(random_seed)

torch.cuda.manual_seed_all(random_seed) # multi-GPU

In [3]:
test = torch.tensor([[0.9408, 0.1332],
        [0.9346, 0.5936],
        [0.8694, 0.5677],
        [0.7411, 0.4294]])

In [11]:
class Net(nn.Module):
    def __init__(self):

        super(Net, self).__init__()
        
#         setattr(self, '123123', nn.Sequential())
        self.layers=nn.Sequential(nn.Linear(2,4),nn.Sequential(nn.Linear(4,2),nn.Linear(4,2)))
#         self.tmp = nn.Linear(2,4)
#         self.tmp1 = nn.Linear(4,2)
#         self.tmp2 = nn.Linear(4,2)
#         method = getattr(self, self.name)
#         a=getattr(self, '123123')
#         a.add_module('456',nn.Linear(4,2))
    def forward(self, x):
        out = self.layers(nn.Linear(2,4)(x),nn.Sequential(nn.Linear(4,2),nn.Linear(4,2)))
#         out = [self.tmp1(self.tmp(x)),self.tmp2(self.tmp(x))]
        out = torch.cat(out)
        return out

In [12]:
seed= 1234
torch.manual_seed(seed)
a=Net()

In [41]:
for i in a.state_dict():
    print(i)

layers.0.weight
layers.0.bias
layers.1.0.weight
layers.1.0.bias
layers.1.1.weight
layers.1.1.bias


In [14]:
# test1
a.forward(test)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (4x2 and 4x2)

In [33]:
a=['parallel',
            ['conv',{'ksize':1, 'chn':64}],
         
            ['serial',
                ['conv',{'ksize':1,'chn':48}],
                ['conv',{'ksize':5, 'chn':64}]],
         
            ['serial',
                ['conv',{'ksize':1,'chn':64}],
                 ['conv',{'ksize':3,'chn':96}],
                 ['conv',{'ksize':3,'chn':96}]],
         
            ['serial',
                ['avg',{'ksize':3,'stride':1}],
                ['conv',{'ksize':1,'chn':'#chn'}]]
        
        ]

In [215]:
# class modules(nn.Module):
#     def __init__(self, layers):
        
#         super(modules, self).__init__()
        
#         self.layer1 = nn.Sequential(
#             *layers
#                                     )
        
        
#     def forward(self, x):
        
#         out = self.layer1(x)
        
#         return out

In [30]:
class mk_modules(nn.Module):
    def __init__(self, layers):
        super(mk_modules, self).__init__()
        
        self.name, layer = layers

        setattr(self, self.name, nn.Sequential(*layer))
        
#         print(layer)
    def forward(self, x ):
        
        method = getattr(self, self.name)
        out = method(x)
        
        return out

In [31]:
class Net(nn.Module):
    def __init__(self, tmp):
#         tmp=tmp
#         print(type(tmp),len(tmp))
        super(Net, self).__init__()
#         print(tmp.__class__.__name__)
#         tmp.__class__.
        c=[]
        for i in tmp:
            for j in i.children():
                
#                 print(j)
                c.append(j)
#                 break
#             self.name , module = i
        self.layer1 = nn.Sequential(
            *c
                            )
        print(self.layer1[0])
        
    def forward(self,x):
        
        out1 = self.layer1[0](x)
        out2 = self.layer1[1](x)
        out3 = self.layer1[2](x)
        out=torch.cat([out1,out2,out3],dim=0)
        
        return out
    
    
    

In [27]:
class Net(nn.Module):
    def __init__(self, a):
        super(Net, self).__init__()

        self.layer1 = nn.Sequential(
            *a
                            )
        
    def forward(self,x):
        out = []
        for i in self.layer1.children():
            out.append(i(x))
#         out1 = self.layer1
        cat=torch.cat(out,dim=0)
        
        return cat

In [32]:
tmp = []
inchannels = 3
# for i in range(2):
layer = ('_1x1',[nn.Conv2d(in_channels=inchannels,out_channels=24,kernel_size=1)])
# tmptmp=mk_modules(layer)
tmp.append(layer)
# tmp.append(mk_modules(layer))

layer = ('_5x5',[nn.Conv2d(in_channels=inchannels,out_channels=16,kernel_size=1),
               nn.Conv2d(in_channels=16,out_channels=24,kernel_size=5,padding=2)])
tmp.append(layer)
tmp.append(mk_modules(layer))

layer = ('_3x3',[nn.Conv2d(in_channels=inchannels,out_channels=16,kernel_size=1),
               nn.Conv2d(in_channels=16,out_channels=24,kernel_size=3,padding=1)])
tmp.append(layer)
# tmp.append(mk_modules(layer))
# print(tmp)

# layer = ('seq',[nn.Conv2d(in_channels=inchannels,out_channels=16,kernel_size=1),
#                nn.Conv2d(in_channels=16,out_channels=24,kernel_size=3,padding=1)])
# tmp.append(mk_modules(layer))

# x=mk_modules(layer)


a=Net(tmp)
x=torch.rand([10,3,299,299])
z=a(x)

Sequential(
  (0): Conv2d(3, 24, kernel_size=(1, 1), stride=(1, 1))
)


In [17]:
for i in a.children():
    print(i)

Sequential(
  (0): mk_modules(
    (_1x1): Sequential(
      (0): Conv2d(3, 24, kernel_size=(1, 1), stride=(1, 1))
    )
  )
  (1): mk_modules(
    (_5x5): Sequential(
      (0): Conv2d(3, 16, kernel_size=(1, 1), stride=(1, 1))
      (1): Conv2d(16, 24, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    )
  )
  (2): mk_modules(
    (_3x3): Sequential(
      (0): Conv2d(3, 16, kernel_size=(1, 1), stride=(1, 1))
      (1): Conv2d(16, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
)


In [5]:
tmp

[mk_modules(
   (_1x1): Sequential(
     (0): Conv2d(3, 24, kernel_size=(1, 1), stride=(1, 1))
   )
 ),
 mk_modules(
   (_5x5): Sequential(
     (0): Conv2d(3, 16, kernel_size=(1, 1), stride=(1, 1))
     (1): Conv2d(16, 24, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
   )
 ),
 mk_modules(
   (_3x3): Sequential(
     (0): Conv2d(3, 16, kernel_size=(1, 1), stride=(1, 1))
     (1): Conv2d(16, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   )
 )]

In [54]:
class model(nn.Module):
    def __init__(self):
        super(model, self).__init__()
        
        self.layer123 = nn.Sequential()
        
    def forward(self,x):
        output= []
        for i in self.layer123:
            output.append(i(x))
            
        cat=torch.cat(output,dim=1)
        
        return cat
            
# model = nn.Sequential()
a= model()
a.layer123.add_module('conv0', tmp[0])
a.layer123.add_module('norm0', tmp[1])
a.layer123.add_module('norm1', tmp[2])

# for i in model.children():
#     print(i)
#     break

In [55]:
class Net(nn.Module):
    def __init__(self,layer):
        super(Net, self).__init__()
        self.layer3 =nn.Sequential(*layer)
        
    def forward(self, x):
        
        out33 = self.layer3(x)
        
        return out33

In [56]:
x=torch.rand([10,3,299,299])
b=Net([a])
z=b(x)

In [57]:
z.shape

torch.Size([10, 72, 299, 299])

In [71]:
import torch

import numpy as np

import random



random_seed = 42



torch.manual_seed(random_seed)

torch.backends.cudnn.deterministic = True

torch.backends.cudnn.benchmark = False



np.random.seed(random_seed)

random.seed(random_seed)



torch.cuda.manual_seed(random_seed)

torch.cuda.manual_seed_all(random_seed) # multi-GPU

In [104]:
test = torch.tensor([[0.9408, 0.1332],
        [0.9346, 0.5936],
        [0.8694, 0.5677],
        [0.7411, 0.4294]])

In [140]:
class Net(nn.Module):
    def __init__(self):

        super(Net, self).__init__()
        
        setattr(self, '123123', nn.Sequential())
#         self.layers=nn.Sequential()
        self.tmp = nn.Linear(2,4)
        self.tmp1 = nn.Linear(4,2)
        self.tmp2 = nn.Linear(4,2)
#         method = getattr(self, self.name)
        a=getattr(self, '123123')
        a.add_module('456',nn.Linear(4,2))
    def forward(self, x):
#         out = self.tmp(x)
        out = [self.tmp1(self.tmp(x)),self.tmp2(self.tmp(x))]
        out = torch.cat(out)
        return out

In [157]:
class Net(nn.Module):
    def __init__(self):

        super(Net, self).__init__()
        
#         setattr(self, '123123', nn.Sequential())
        self.layers=nn.Sequential(nn.Linear(2,4))
        self.layers.add_module(nn.Sequential(nn.Linear(4,2),nn.Linear(4,2)))
#         self.tmp = nn.Linear(2,4)
#         self.tmp1 = nn.Linear(4,2)
#         self.tmp2 = nn.Linear(4,2)
#         method = getattr(self, self.name)
#         a=getattr(self, '123123')
#         a.add_module('456',nn.Linear(4,2))
    def forward(self, x):
        out = self.layers(x)
#         out = [self.tmp1(self.tmp(x)),self.tmp2(self.tmp(x))]
        out = torch.cat(out)
        return out

In [158]:
seed= 1234
torch.manual_seed(seed)
a=Net()

TypeError: list is not a Module subclass

In [155]:
a

Net(
  (layers): Sequential(
    (0): Linear(in_features=2, out_features=4, bias=True)
  )
  (layers1): Sequential(
    (0): Linear(in_features=4, out_features=2, bias=True)
    (1): Linear(in_features=4, out_features=2, bias=True)
  )
)

In [152]:
a.tmp.weight

AttributeError: 'Net' object has no attribute 'tmp'

In [103]:
# test1
a.forward(test)

tensor([[-0.2560, -0.2805],
        [-0.3323, -0.3199],
        [-0.3104, -0.3247],
        [-0.2524, -0.3269],
        [-0.0037,  0.0038],
        [ 0.0587,  0.0528],
        [ 0.0764,  0.0446],
        [ 0.0998,  0.0191]], grad_fn=<CatBackward>)

In [108]:
a.forward(test)

tensor([[-0.2560, -0.2805],
        [-0.3323, -0.3199],
        [-0.3104, -0.3247],
        [-0.2524, -0.3269],
        [-0.0037,  0.0038],
        [ 0.0587,  0.0528],
        [ 0.0764,  0.0446],
        [ 0.0998,  0.0191]], grad_fn=<CatBackward>)

In [156]:
a.forward(test)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (4x2 and 4x2)

In [23]:
list(a.layers.named_children())

[('namename',
  Sequential(
    (conv1): Conv2d(3, 2, kernel_size=(2, 2), stride=(1, 1))
    (conv): Conv2d(3, 2, kernel_size=(2, 2), stride=(1, 1))
  ))]

In [56]:
np.prod(np.array([2,3]))

6

In [63]:
torch.tensor(torch.rand([3,5]).shape)

tensor([3, 5])

In [58]:
torch.prod(torch.tensor([2,6]))

tensor(12)

In [190]:
a.layer1._modules['0']

mk_modules(
  (_1x1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(1, 1), stride=(1, 1))
  )
)

In [141]:
a.forward

['__call__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__func__', '__ge__', '__get__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__self__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']
['__call__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__func__', '__ge__', '__get__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__self__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']
['__call__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__func__', '__ge__', '__get__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__self__', '__seta

In [102]:
name = 'd'
d='f'
s=globals()['{}'.format(name)]
s

'f'

In [31]:
b

Net(
  (layer1): Sequential(
    (0): Sequential(
      (0): Linear(in_features=2, out_features=3, bias=True)
      (1): Conv2d(3, 6, kernel_size=(2, 2), stride=(1, 1))
      (2): Linear(in_features=6, out_features=3, bias=True)
    )
    (1): Sequential(
      (0): Linear(in_features=2, out_features=3, bias=True)
      (1): Conv2d(3, 6, kernel_size=(2, 2), stride=(1, 1))
      (2): Linear(in_features=6, out_features=3, bias=True)
    )
  )
)

In [10]:
for i in layers:
    if isinstance(i, nn.Linear):
        print(i.weight.shape)

torch.Size([3, 2])


In [14]:
if isinstance(layers, nn.Linear):
    print(layers.weight.shape)

In [ ]:
class net(nn.Module):
    def __init__(self):


SyntaxError: invalid syntax (Temp/ipykernel_7120/953631979.py, line 2)

In [94]:
class tmp(nn.Module):
    def __init__(self,a):
        super(tmp, self).__init__()
        self.net=nn.Sequential()
        self.net.add_module('dict',nn.ModuleDict(a))
    def forward(self,x):
#         a=getattr(self, '123123')
#         a.add_module('456',nn.Linear(4,2))
        out=getattr(self,'net')
        out=out.dict['a'](x)
        return out

In [95]:
di={'a':nn.Linear(2,4)}

In [96]:
n=tmp(di)
n

tmp(
  (net): Sequential(
    (dict): ModuleDict(
      (a): Linear(in_features=2, out_features=4, bias=True)
    )
  )
)

In [27]:
test = torch.tensor([[0.9408, 0.1332],
        [0.9346, 0.5936],
        [0.8694, 0.5677],
        [0.7411, 0.4294]])

In [98]:
n(test)

tensor([[-0.7961, -0.4668, -0.4197, -0.1295],
        [-0.9792, -0.4800, -0.1581,  0.0325],
        [-0.9442, -0.4521, -0.1319,  0.0222],
        [-0.8402, -0.3943, -0.1292, -0.0289]], grad_fn=<AddmmBackward>)

In [25]:
import torch
import torch.nn as nn
class A(nn.Module):
    def __init__(self,layers):
        super(A, self).__init__()
        self.layer=nn.Sequential(nn.Linear(2,4),*layers)
    def forward(self,x):
        out = self.layer(x)
        return out

In [19]:
class B(nn.Module):
    def __init__(self):
        super(B, self).__init__()
        self.layer=nn.Linear(4,2)
    def forward(self,x):
        out = self.layer(x)
        return out

In [36]:
class B(nn.Module):
    def __init__(self):
        super(B, self).__init__()
        self.layer=nn.Linear(2,1)
        self.layer1=nn.Linear(2,1)
        
    def forward(self,x):
        out = self.layer(x)
        out1 = self.layer1(x)
        out = torch.cat([out,out1],dim=1)
        return out

In [20]:
b=B()

In [37]:
b_b=B()

In [38]:
a=A([b,b_b])

In [39]:
test.shape

torch.Size([4, 2])

In [40]:
a

A(
  (layer): Sequential(
    (0): Linear(in_features=2, out_features=4, bias=True)
    (1): B(
      (layer): Linear(in_features=4, out_features=2, bias=True)
    )
    (2): B(
      (layer): Linear(in_features=2, out_features=1, bias=True)
      (layer1): Linear(in_features=2, out_features=1, bias=True)
    )
  )
)

In [41]:
a(test)

tensor([[ 0.0076,  0.5564],
        [-0.0042,  0.5767],
        [-0.0118,  0.5817],
        [-0.0247,  0.5877]], grad_fn=<CatBackward0>)